In [1]:
import pandas as pd
from PreprocessingUtils import prepare_raw_data, count_distances

In [2]:
data = pd.read_parquet('../data/raw_data_delays.parquet')
gps = pd.read_csv('../data/station_gps.csv')

In [3]:
data = prepare_raw_data(data)
data = pd.merge(data, gps, on='Stacja', how='left')

### station count
<ul>
    <li>full route station count</li>
    <li>station count on current station</li>
</ul>

In [4]:
data['station_count_on_curr_station'] = data.groupby(['pk', 'Relacja']).cumcount()
data['full_route_station_count'] = data.groupby(['pk', 'Relacja'])['Relacja'].transform('count')

### distances
<ul>
    <li>full route distance</li>
    <li>distance distance until current station</li>
    <li>distance from start station</li>
    <li>distance to final station</li>
    <li>distance distance from the nearest big city station</li>
</ul>

In [36]:
# pks = data['pk'].unique()[:1]
# dfs_list = []

In [37]:
# big_city_names = pd.read_csv('../data/big_cities_dict.csv', sep=';')['miasto'].unique()
# big_city_stations_gps = gps[gps['Stacja'].isin(big_city_names)].reset_index(drop=True).copy()
# 
# for pk in pks:
#     temp_df = data[data['pk']==pk].reset_index(drop=True).copy()
#     distance_to_prev_station_list = []
#     for i in range(len(temp_df)): 
#         curr_gps = (temp_df.loc[i, 'lat'].item(), temp_df.loc[i, 'lon'].item())
#         dist_to_near_city_list = []
#         
#         for j in range(len(big_city_stations_gps)):
#             curr_city_gps = (big_city_stations_gps.loc[j, 'lat'], big_city_stations_gps.loc[j, 'lon'])
#             dist_to_near_city = haversine(curr_gps, curr_city_gps)
#             dist_to_near_city_list.append(dist_to_near_city)
# 
#         pos = np.array(dist_to_near_city_list).argmin()
#         temp_df.loc[i, 'distance_to_near_city_station'] = dist_to_near_city_list[pos]
#         temp_df.loc[i, 'near_city_station_name'] = big_city_stations_gps.loc[pos, 'Stacja']
#         
#         if i == 0:
#             distance_to_prev_station_list.append(0)
#         else:
#             prev_gps = (temp_df.loc[i-1, 'lat'].item(), temp_df.loc[i-1, 'lon'].item())
#             distance = haversine(curr_gps, prev_gps, unit=Unit.KILOMETERS)
#             distance_to_prev_station_list.append(distance)
# 
#     temp_df['distance_to_prev_station'] = distance_to_prev_station_list
#     temp_df['distance_from_start'] = temp_df['distance_to_prev_station'].cumsum()
#     temp_df['distance_to_final'] = np.array(temp_df['distance_from_start'][::-1])
#     temp_df['full_route_distance'] = temp_df.groupby('pk')['distance_to_prev_station'].transform('sum')
# 
#     dfs_list.append(temp_df)
#     print(pk)
# 
# new_data = pd.concat(dfs_list, ignore_index=True) 

1


In [ ]:
data = count_distances(data, gps)

In [6]:
data

,pk,Relacja,Stacja,Opóźnienie przyjazdu,Opóźnienie odjazdu,arrival_on_time,departure_on_time,lat,lon,station_count_on_curr_station,full_route_station_count,distance_to_near_city_station,near_city_station_name,distance_to_prev_station,distance_from_start,distance_to_final,full_route_distance
0,1,Przeworsk - Rzeszów Główny,Przeworsk,0,0,2022-12-01 06:55:00,2022-12-01 06:55:00,50.067161,22.503232,0,10,35.612095,Rzeszów Główny,0.000000,0.000000,36.484418,36.484418
1,1,Przeworsk - Rzeszów Główny,Grzęska,0,0,2022-12-01 06:58:30,2022-12-01 06:59:00,50.080037,22.452564,1,10,32.152599,Rzeszów Główny,3.889077,3.889077,31.639766,36.484418
2,1,Przeworsk - Rzeszów Główny,Rogóżno koło Łańcuta,0,0,2022-12-01 07:03:00,2022-12-01 07:03:30,50.083573,22.362513,2,10,25.854248,Rzeszów Główny,6.437439,10.326516,28.265814,36.484418
3,1,Przeworsk - Rzeszów Główny,Kosina,0,0,2022-12-01 07:06:30,2022-12-01 07:07:00,50.083570,22.322846,3,10,23.070959,Rzeszów Główny,2.830260,13.156776,22.842960,36.484418
4,1,Przeworsk - Rzeszów Główny,Głuchów,0,0,2022-12-01 07:09:30,2022-12-01 07:10:00,50.083469,22.279077,4,10,20.013113,Rzeszów Główny,3.122964,16.279739,20.043487,36.484418
5,1,Przeworsk - Rzeszów Główny,Łańcut,0,0,2022-12-01 07:14:00,2022-12-01 07:14:30,50.081750,22.226396,5,10,16.318646,Rzeszów Główny,3.763748,20.043487,16.279739,36.484418
6,1,Przeworsk - Rzeszów Główny,Krzemienica,0,0,2022-12-01 07:17:00,2022-12-01 07:17:30,50.081738,22.187162,6,10,13.635759,Rzeszów Główny,2.799473,22.842960,13.156776,36.484418
7,1,Przeworsk - Rzeszów Główny,Strażów,0,0,2022-12-01 07:21:00,2022-12-01 07:21:30,50.067424,22.114520,7,10,8.214880,Rzeszów Główny,5.422854,28.265814,10.326516,36.484418
8,1,Przeworsk - Rzeszów Główny,Rzeszów Załęże,0,0,2022-12-01 07:24:00,2022-12-01 07:24:30,50.058533,22.069328,8,10,4.844652,Rzeszów Główny,3.373952,31.639766,3.889077,36.484418
9,1,Przeworsk - Rzeszów Główny,Rzeszów Główny,0,0,2022-12-01 07:29:00,2022-12-01 07:29:00,50.043213,22.005808,9,10,0.000000,Rzeszów Główny,4.844652,36.484418,0.000000,36.484418


### longer stop duration

In [13]:
data['stop_duration'] = (data['departure_on_time'] - data['arrival_on_time']).dt.total_seconds()/60

In [19]:
data[data['stop_duration']

,pk,Relacja,Stacja,Opóźnienie przyjazdu,Opóźnienie odjazdu,arrival_on_time,departure_on_time,lat,lon,station_count,how_many_stations,stop_duration
114990,5076,Warszawa Wschodnia - Bohumin,Katowice,8,7,2022-11-20 23:58:00,2022-11-20 00:02:00,50.257607,19.017146,10,15,-1436.0
115051,5079,Kraków Główny - Gdynia Główna,Wrocław Główny,14,5,2022-11-20 23:27:00,2022-11-20 00:07:00,51.097968,17.037530,13,31,-1400.0
120002,5289,Frankfurt/Oder - Warszawa Wschodnia,Warszawa Centralna,2,2,2022-11-24 23:53:00,2022-11-24 00:01:00,52.228833,21.002915,8,10,-1432.0
120040,5290,Katowice - Zwardoń,Sól Kiczora,1,1,2022-11-24 23:59:30,2022-11-24 00:01:00,49.517164,19.031315,36,39,-1438.5
120102,5292,Wrocław Główny - Przemyśl Główny,Radymno,1,1,2022-11-24 23:59:30,2022-11-24 00:00:30,49.939721,22.828321,23,26,-1439.0
...,...,...,...,...,...,...,...,...,...,...,...,...
959711,43731,Zakopane - Gdynia Główna,Kraków Główny,0,0,2022-12-30 23:36:00,2022-12-30 00:06:00,50.068666,19.947802,7,32,-1410.0
961262,43807,Zakopane - Gdynia Główna,Kraków Główny,17,6,2022-12-31 23:36:00,2022-12-31 00:06:00,50.068666,19.947802,7,32,-1410.0
1073062,49122,Rzeszów Główny - Przeworsk,Głuchów,0,0,2023-04-22 23:59:36,2023-04-22 00:00:06,50.083469,22.279077,5,10,-1439.5
1073129,49127,Wejherowo - Gdańsk Główny,Gdynia Leszczynki,1,1,2023-04-22 23:59:42,2023-04-22 00:00:06,54.541340,18.478510,9,27,-1439.6
